# CSV data

**TODO**: Convert to using [csvkit](https://csvkit.readthedocs.io/en/1.0.2/) Much better!


## Exporting data from Excel

Data of interest that we want to process in Python often comes in the form of an Excel spreadsheet. The easiest way to get access to it is to export the data in CSV format.

Let's get some data. Download [Sample Superstore Sales .xls file](https://community.tableau.com/docs/DOC-1236) or [my local copy](https://github.com/parrt/msan692/raw/master/data/SampleSuperstoreSales.xls) and open it in Excel. Select `File>Save As...` menu option and then "Comma separated values (.csv)" from the `Format:` drop-down menu. Set the filename to `SampleSuperstoreSales.csv` or similiar. That will warn you that the data cannot be saved as CSV without losing some information. Don't worry about that because it's only formatting and not data information that you will lose. Say "Save Active Sheet". Then it will helpfully give you a second warning. Tell it to continue.

If you use `more` from the commandline, you will see some funny characters in the output:

<img src="figures/csv-funny-char.png" style="width:600px">

That weird `<A8>` character is character code 168 (or A8 in hexadecimal), which is the registered trademark symbol &#x00AE;.  From experience with these data formats, I know that Excel is saving things using a "Latin-1" text encoding (which allows 0..255 char values and defined accented characters and other special symbols beyond ASCII). That encoding is essentially ASCII but anything above code 127 is Euro-specific. So, we need to fix that by stripping those out. 

We can accomplish that using the [iconv](https://www.gnu.org/software/libiconv/) command from the terminal:

In [8]:
! iconv -c -f latin1 -t ascii data/SampleSuperstoreSales.csv > /tmp/t.csv

We could also pretend that it is encoded as UTF-8, which also allows the Latin-1 character set:

```bash
$ iconv -c -f utf-8 -t ascii SampleSuperstoreSales.csv > /tmp/t.csv
```

Ok, now we finally have some plain CSV data in `/tmp/t.csv` that we can pull into Python:

In [9]:
! head -4 /tmp/t.csv

##  Dealing with commas double quotes in CSV

For the most part, CSV files are very simple, but they can get complicated when we need to embed a comma. One such case from the above file shows how fields with commas get quoted:

```
"Eldon Base for stackable storage shelf, platinum"
```

What happens when we want to encode a quote? Well, somehow people decided that `""` double quotes was the answer (not!) and we get fields encoded like this:

```
"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators"
```

The good news is that Python's `csv` package knows how to read Excel-generated files that use such encoding. Here's a sample script that reads such a file into a list of lists:

In [2]:
import sys
import csv
print "hi"
table_file = "data/SampleSuperstoreSales.csv"
with open(table_file, "rb") as csvfile:
    f = csv.reader(csvfile, dialect='excel')
    data = []
    for row in f:
        data.append(row)

print data[0]
print data[1]

hi
['Row ID', 'Order ID', 'Order Date', 'Order Priority', 'Order Quantity', 'Sales', 'Discount', 'Ship Mode', 'Profit', 'Unit Price', 'Shipping Cost', 'Customer Name', 'Province', 'Region', 'Customer Segment', 'Product Category', 'Product Sub-Category', 'Product Name', 'Product Container', 'Product Base Margin', 'Ship Date']
['1', '3', '10/13/10', 'Low', '6', '261.54', '0.04', 'Regular Air', '-213.25', '38.94', '35', 'Muhammed MacIntyre', 'Nunavut', 'Nunavut', 'Small Business', 'Office Supplies', 'Storage & Organization', 'Eldon Base for stackable storage shelf, platinum', 'Large Box', '0.8', '10/20/10']


We can extend that a little bit if we want that data in a numpy `array` (See the full [csv2numpy.py](https://github.com/parrt/msan692/blob/master/notes/code/csv2numpy.py)):

In [5]:
import numpy as np
npdata = np.array(data)
print type(npdata)
print npdata.shape # print the dimensions
print npdata

<type 'numpy.ndarray'>
(8400, 21)
[['Row ID' 'Order ID' 'Order Date' ..., 'Product Container'
  'Product Base Margin' 'Ship Date']
 ['1' '3' '10/13/10' ..., 'Large Box' '0.8' '10/20/10']
 ['49' '293' '10/1/12' ..., 'Jumbo Drum' '0.58' '10/2/12']
 ..., 
 ['7906' '56550' '4/8/11' ..., 'Small Pack' '0.41' '4/10/11']
 ['7907' '56550' '4/8/11' ..., 'Small Box' '0.56' '4/9/11']
 ['7914' '56581' '2/8/09' ..., 'Medium Box' '0.65' '2/11/09']]


## Exercise

Extract the quantity and unit price columns and multiply them together to get the sale value. My solution uses a list comprehension across the list of lists, one per column I need. Then I create a numpy array of those and simply multiply them with `*`. Remember that `data[1:]` gives you all but the first element (a row in this case) of a list. `float(x)` converts string or integer `x` to a floating point number. If you get stuck, see [readcsv.py](https://github.com/parrt/msan692/blob/master/notes/code/readcsv.py).

## Pandas Data frames

In the end, the easiest way to deal with loading CSV files is probably with [Pandas](http://pandas.pydata.org/). For example, to load our sales CSV, we don't even have to manually open and close a file:

In [12]:
import pandas
pandas.options.display.max_rows = 4 # Don't display too much data (Pandas)
table = pandas.read_csv("data/SampleSuperstoreSales.csv")
table

,Row ID,Order ID,Order Date,Order Priority,Order Quantity,Sales,Discount,Ship Mode,Profit,Unit Price,...,Customer Name,Province,Region,Customer Segment,Product Category,Product Sub-Category,Product Name,Product Container,Product Base Margin,Ship Date
0,1,3,10/13/10,Low,6,261.5400,0.04,Regular Air,-213.25,38.94,...,Muhammed MacIntyre,Nunavut,Nunavut,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,0.80,10/20/10
1,49,293,10/1/12,High,49,10123.0200,0.07,Delivery Truck,457.81,208.16,...,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Appliances,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Jumbo Drum,0.58,10/2/12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8397,7907,56550,4/8/11,Not Specified,8,469.8375,0.00,Regular Air,-159.24,65.99,...,Frank Hawley,Alberta,West,Home Office,Technology,Telephones and Communication,Talkabout T8367,Small Box,0.56,4/9/11
8398,7914,56581,2/8/09,High,20,2026.0100,0.10,Express Air,580.43,105.98,...,Grant Donatelli,Alberta,West,Consumer,Furniture,Office Furnishings,"Tenex 46"" x 60"" Computer Anti-Static Chairmat,...",Medium Box,0.65,2/11/09


Pandas hides all of the details. I also find that pulling out columns is nice with pandas. Here's how to print the customer name column:

In [13]:
names = table['Customer Name']
names

0       Muhammed MacIntyre
1             Barry French
               ...        
8397          Frank Hawley
8398       Grant Donatelli
Name: Customer Name, dtype: object

I also like to convert this to a numpy matrix so that I can pull out various rows conveniently:

In [14]:
m = table.as_matrix()
print m[0:2]

[[1 3 '10/13/10' 'Low' 6 261.54 0.04 'Regular Air' -213.25 38.94 35.0
  'Muhammed MacIntyre' 'Nunavut' 'Nunavut' 'Small Business'
  'Office Supplies' 'Storage & Organization'
  'Eldon Base for stackable storage shelf, platinum' 'Large Box' 0.8
  '10/20/10']
 [49 293 '10/1/12' 'High' 49 10123.02 0.07 'Delivery Truck' 457.81 208.16
  68.02 'Barry French' 'Nunavut' 'Nunavut' 'Consumer' 'Office Supplies'
  'Appliances' '1.7 Cubic Foot Compact "Cube" Office Refrigerators'
  'Jumbo Drum' 0.58 '10/2/12']]


You can learn more about [slicing and dicing data](https://github.com/parrt/msan501/blob/master/notes/data.ipynb) from our Boot Camp notes.

### Exercise

From the sales CSV file on the net and use pandas to read in the data adn extract the `Profit` and `Unit Price` columns into a new data frame. Hint: You can use a list of column names like `['Profit','Unit Price']` as a data frame index.

In [15]:
table = pandas.read_csv("data/SampleSuperstoreSales.csv")
table[['Profit','Unit Price']]

,Profit,Unit Price
0,-213.25,38.94
1,457.81,208.16
...,...,...
8397,-159.24,65.99
8398,580.43,105.98
